In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

c:\Users\prave\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.ImageFolder(root='./FACE', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):         
        super(Net, self).__init__()
        self.fc1 = nn.Linear(150528, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
      
        x = x.view(-1,150528)
        
        x = F.relu(self.fc1(x))
        return x

In [5]:
input_tensor = torch.randn(1, 3, 224, 224)

model = Net().to(device)

In [6]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [7]:
num_epochs = 1

In [8]:
train_loss_list = []
output_dist_list = []

In [9]:
writer=SummaryWriter('')

In [10]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / len(trainloader)))
            # TensorBoard
    writer.add_scalar('Training Loss', running_loss / 100, epoch * len(trainloader) + i)

    
    with torch.no_grad():
        input_tensor = torch.randn(1, 3, 224, 224)
        
        output_np = outputs.cpu().detach().numpy() 
        output_dist_list.append(output_np)

    writer.add_histogram('Output Distribution', output_np, epoch)
    
writer.close()

Epoch 1 loss: 0.644


In [11]:
images, labels = next(iter(trainloader))

grid = torchvision.utils.make_grid(images)
writer.add_image('images', grid, 0)


In [15]:
input_tensor = torch.randn(1, 3, 224, 224)
mo=Net()
writer.add_graph(mo, input_tensor)


writer.close()

In [16]:
%reload_ext tensorboard

In [18]:
import tensorboard
%tensorboard --logdir=runs --port=7000
